## 이 process module version2는 전문가 답변을 필터링하는 5번 함수가 없습니다. 
## 기존에 크롤링 시 전문가 답변을 한 번 필터링 했기 때문에 필요 없기 때문입니다.

<br>

# <데이터 전처리 함수 정리>
### 1. load_data : 데이터를 로드하고 결측치를 제거하는 함수
- input : None
- output : 전체 데이터셋, 질문 리스트, 답변 리스트

### 2. clean_text: 질문과 답변 리스트에 대해 통합 전처리를 진행하는 함수
- input : list (question or answer)
- output : filtered list

### 3. sentence_split : 질문과 답변에 대해 kss로 문장을 분리하는 함수
- input : list (question or answer)
- output : seperated list

### 4. clean_question : 질문 맞춤형 전처리 함수
- input : question list
- output : filtered question list

### 5. clean_psy_answer : 전문의 답변 맞춤형 전처리 함수
- input : psy answer list
- output : filtered psy answer list

### 6. custom_clean_answr : 전문의 답변에 대해 커스텀 필터링
- input : psy answer list
- output : filtered pasy answer list

### 7. save_data : 데이터를 dataframe 형태로 만든 후, csv로 저장하는 함수
- input : questions, annswers, index, filename
- output : dataframe






# 0. Import Pakages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
cd /content/drive/MyDrive/TobigSlang

/content/drive/.shortcut-targets-by-id/1Tys5-QaX-3ICj4J5S283IJYXj7FSQFAe/TobigSlang


In [ ]:
cd '2. Modeling '/'정진모'/'미경'/models/'#preprocessing'

/content/drive/.shortcut-targets-by-id/1Tys5-QaX-3ICj4J5S283IJYXj7FSQFAe/TobigSlang/2. Modeling /정진모/미경/models/#preprocessing


In [ ]:
ls

'1227data(1)'/  'crawling_1227_colab(2).ipynb'   kin_1228.csv
 backup/         data/                           preprocess.ipynb
 chromedriver    kin_1227.csv                    preprocess_module2.ipynb


In [ ]:
#kss 설치
!sudo apt install python3-dev
!pip install kss

import numpy as np
import pandas as pd
import re
import math
import kss
from tqdm import tqdm_notebook

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


# 1. 데이터 로드 및 결측치 제거

In [ ]:
#load_data : 데이터를 불러오고, 결측값을 제거합니다.
def load_data():
  kin = pd.read_csv("kin_1228.csv", encoding='utf-8')
  print("Data load success! \nTotal row is", len(kin))

  #remove missing value 
  kin = kin.dropna(axis=0).reset_index(drop=True)
  print("\nAfter removing missing values, the total number of rows is ", len(kin))

  #split data
  questions = kin['question']
  answers = kin['answer']

  return kin, questions, answers

In [ ]:
kin, questions, answers = load_data()

Data load success! 
Total row is 33305

After removing missing values, the total number of rows is  33264


# 2. Data Cleansing
question과 answer에 대해 공통적으로 cleansing 수행.

1. 기본적인 특수문자를 제거
2. kss로 문장을 분리

## 2-1. 특수문자 제거

In [ ]:
def clean_text(texts):
  removed_list = []
  for i in range(0, len(texts)): 
    text = re.sub(r'[@%\\*=()/~&\+á?\xc3\xa1\-\|\:\;\!\-\,\_\~\$\'\"\“]', '',str(texts[i])) #remove punctuation 
    text = re.sub(r'\d+','', text)# remove number
    text = re.sub(r'\.\.+', '.', text) #점이 두 개 이상 반복되는 부분 제거
    text = re.sub(r'\u200B', '', text) #폭 없는 공백 제거
    text = text.lower() #lower case 
    text = re.sub(r'\s+', ' ', text) #remove extra space 
    text = re.sub(r'<[^>]+>','',text) #remove Html tags
    text = re.sub(r'\s+', ' ', text) #remove spaces
    text = re.sub(r"^\s+", '', text) #remove space from start 
    text = re.sub(r'\s+$', '', text) #remove space from the end
    text = re.sub(r'\.', '', text) #점 제거
    removed_list.append(text)

  return removed_list

In [ ]:
c_questions = clean_text(questions)
c_answers = clean_text(answers)

## 2-2. kss로 문장 분리

In [ ]:
def sentence_split(texts):
  results = []
  for i in tqdm_notebook(range(len(texts))):
    sen = []
    s = texts[i]
    # print('[분리 전]:', s)
    for sent in kss.split_sentences(s):
      sen.append(sent)
    result = '. '.join(sen)
    # print('[분리 후]:', result)
    results.append(result)
  return results

In [ ]:
s_questions = sentence_split(c_questions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
s_answers = sentence_split(c_answers)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
  kin_dic = {'question': s_questions, 'answer': s_answers}
  kin_kss = pd.DataFrame(data=kin_dic)

In [ ]:
# 혹시 모르니까 한번 저장할 것
kin_kss.to_csv("kin_kss.csv", index=False)

In [ ]:
# 불러와서 새로 진행
kin_kss = pd.read_csv("kin_kss.csv", encoding='utf-8')

In [ ]:
kin_kss

,question,answer
0,아버지께서 코골이가 심하십니다. 코골이소리 엄청 큰 것은 물론이고요. 이번에 본집가...,안녕하세요. 닥톡네이버 지식in 상담의사 신홍범입니다. 수면무호흡증의 위험성과 관련...
1,꿈을 안꾸는 날이 년에 손꼽을 정도이고 매일 꿈을 꿉니다. 꿈들이 다 기억이 나며 ...,안녕하세요. 닥톡 네이버 지식인 상담의사 신홍범입니다. 저는 매일 밤 꿈을 꿉니다....
2,잠을 잘 못자요 잠들 때 까지도 시간이 오래걸리지만 잠들고 나서 새벽에 자꾸 깨서 ...,안녕하세요. 닥톡네이버 지식in 상담의사 신홍범입니다. 수면클리닉에 가면 모든 수면...
3,안녕하세요. 곧 살이 될 사람입니다. 다름이 아니라 제가 고등학생 때부터 몽유병인가...,안녕하세요. 닥톡 네이버 상담의사 신홍범 입니다. 몽유병의 치료 방법에 대해서 궁금...
4,불면증 약먹어도 괜찮을까요,안녕하세요. 닥톡 네이버 지식인 상담의사 신홍범입니다. 불면증으로 고생하시는 분들이...
...,...,...
33259,제가 두달 전 공황발작 후 동네 신경정신과에서 처방받은 약을 세네번 먹었구요. 근데...,안녕하세요. 먼저 공황증상에 대해 설명하기 앞서서 불안장애의 큰 분류에 대해서 말씀...
33260,명인벤즈트로핀메실레이트정mg약성분좀 알려주세요,안녕하세요. 질문하신 벤즈트로핀메실레이트정은 이름자체가 성분명입니다. 주로 뇌속의 ...
33261,선천적으로 토를 너무 자주 하는 것 같습니다. 살 여자 술 담배는 전혀 안합니다. ...,안녕하세요. 하이닥네이버 지식in 상담의 이동한 입니다. 잦은 구토 증세로 많이 힘...
33262,살 남자입니다. 일단 저는 제가 우울증이라 생각한적이 없습니다. 커오면서 외로움도 ...,안녕하세요. 잠실하늘 정신건강의학과 박지웅 원장입니다. 마치 점점 추워지는 겨울처럼...


In [ ]:
c_questions = kin_kss['question']
c_answers = kin_kss['answer']

# 3. Question Cleansing

In [ ]:
def clean_question(texts):

  #1. 위에서 공통적으로 보이는 패턴을 수정합니다.
      # a. 앞과 뒤의 white space
      # b. .뒤에 space가 없는 애들은 space 넣어줄 것.
      # c. .앞 뒤로 글자가 없는 애들은 지워줄 것.

  #2. 12/27 민정 수정
      # 1) 문장 끝에 ..이 오는 경우 .으로 수정
      # 2) 문장 중간 혹은 시작할 때 .. 이 오는 경우 제거
      # 3) ㅠㅠ, ㅋㅋ, ㅎㅎ 등 자음만 혹은 모음만 있는 경우 제거
      # 4) … 제거
      # 5) 이모티콘 제거
  
  # float<nan>을 str<nan>으로 변경 (제거는 나중에 한꺼번에)
  for i in range(len(texts)):
    if type(texts[0]) != type(texts[i]):
      print(texts[i])
      texts[i] = str(texts[i])

  remove_pattern = []
  for i in range(0, len(texts)): 
    text = re.sub(r'^\.', '',texts[i]) #맨 앞에 문자가 아닌 것이 등장한 경우
    text = re.sub(r'\s+\.$', '', text) #맨 뒤에 문자가 아닌 것이 등장한 경우 
    text = re.sub(r'\s+\.\s+', '', text) #무의미한 .이 들어있는 경우 - (3)번
    text = re.sub(r"^\s+", '', text) #remove space from start 
    text = re.sub(r'\s+$', '', text) #remove space from the end
    text = re.sub('[|ㄱ-ㅎ|ㅏ-ㅣ]+', '', text) # 3) ㅠㅠ, ㅋㅋ, ㅎㅎ 등 자음만 혹은 모음만 있는 경우 제거
    text = re.sub('…', '', text) # 4) … 제거
    text = re.sub('❌', '', text) # 5) 이모티콘 제거

    # (2)번 마침표 뒤에 스페이스 없는 경우 스페이스 추가.
    while re.search('[가-힣]\.\S', text):
      mats = re.search('\w\.\S', text)
      start = mats.span()[0]
      end = mats.span()[1]
      text = text[:start+2]+' '+text[end-1:]

    # 1) 문장 끝에 ..이 오는 경우 .으로 수정
    while re.search('[가-힣]\.\.\s', text):
      text = re.sub(r'\.\.', '\.', text)

    # 2) 문장 중간 혹은 시작할 때 .. 이 오는 경우 제거
    text = re.sub(r'\.\.', '', text) 

    remove_pattern.append(text)

  print("removing repeated pattern completed.")
  
  print("all cleaning question process is finished.")
  print("length of data is", len(remove_pattern))
  return remove_pattern

In [ ]:
final_questions = clean_question(c_questions)

nan
nan
nan
nan
nan
nan
nan
nan
nan
removing repeated pattern completed.
all cleaning question process is finished.
length of data is 33264


# 4. Answr Cleansing
이 함수에서는 다음과 같은 순서로 답변을 전처리합니다.
1. 전문인 외 답변 제거
2. 인삿말, 해시태그와 출처링크 삭제
4. 기타 문구 제거


## 4-1. 인삿말, 해시태그, 출처 등 삭제

In [ ]:
def clean_psy_answer(texts):

  # float<nan>을 str<nan>으로 변경 (제거는 나중에 한꺼번에)
  for i in range(len(texts)):
    if type(texts[0]) != type(texts[i]):
      print(texts[i])
      texts[i] = str(texts[i])

  #1. remove greeting
  pattern1 = '^안녕하세요.{,50}입니다.'
  pattern2 = '^안녕하십니까.{,50}입니다.'

  remove_greeting1 = []
  for i in range(0, len(texts)): 
    text = re.sub(pattern1, '', texts[i])
    remove_greeting1.append(text)
    
  remove_greeting2 = []
  for i in range(0, len(remove_greeting1)): 
    text = re.sub(pattern2, '', remove_greeting1[i])
    remove_greeting2.append(text)

  print("removing greeting completed.")

  #2. 위에서 공통적으로 보이는 패턴을 수정합니다.
      # a. 앞과 뒤의 white space
      # b. .뒤에 space가 없는 애들은 space 넣어줄 것.
      # c. .앞 뒤로 글자가 없는 애들은 지워줄 것.
  
  remove_pattern = []
  for i in range(0, len(remove_greeting2)): 
    text = re.sub(r'^\.', '',remove_greeting2[i]) #맨 앞에 문자가 아닌 것이 등장한 경우
    text = re.sub(r'\s+\.$', '', text) #맨 뒤에 문자가 아닌 것이 등장한 경우 
    text = re.sub(r'\s+\.\s+', '', text) #무의미한 .이 들어있는 경우 - (3)번
    text = re.sub(r"^\s+", '', text) #remove space from start 
    text = re.sub(r'\s+$', '', text) #remove space from the end

    # (2)번 마침표 뒤에 스페이스 없는 경우 스페이스 추가.
    while re.search('[가-힣]\.\S', text):
      mats = re.search('\w\.\S', text)
      start = mats.span()[0]
      end = mats.span()[1]
      text = text[:start+2]+' '+text[end-1:]
    remove_pattern.append(text)
  print("removing repeated pattern completed.")

  #3. 해시태그, 출처, url 제거
  removed_list = []
  for i in range(0, len(remove_pattern)): 
    text = re.sub(r'#\w+', '',remove_pattern[i]) #해시태그 삭제
    text = re.sub(r'출처', '', text) #출처 단어 삭제
    text = re.sub(r"[a-z]+\.", '', text) #이메일 삭제
    text = re.sub(r"\[.+\]", '', text) #[문자] 형태 삭제
    text = re.sub(r"\s+", ' ', text) #위의 단어들이 삭제되면서 생긴 white space 삭제.
    text = re.sub(r"^\s+", '', text) #remove space from start 
    text = re.sub(r'\s+$', '', text) #remove space from the end
    removed_list.append(text)
    
  print("removing special pattern completed.")

  print("all cleaning answer process is finished.")
  print("length of data is", len(removed_list))
  return removed_list

In [ ]:
c_answers = clean_psy_answer(c_answers)

removing greeting completed.
removing repeated pattern completed.
removing special pattern completed.
all cleaning answer process is finished.
length of data is 33264


## 4-2. 기타 문구 제거
육안으로 확인하면서 반복되는 패턴 등을 추가하여 삭제할 예정

In [ ]:
def custom_clean_answer(texts):
  removed_list = []
  for i in range(0, len(texts)):
    text = re.sub(r'※ 의사 전문가의 예상 치료비 할인 가능 여부 편집된 답변 내용 등 추가 정보가 궁금하시면 아래를 통해 확인할 수 있습니다', '', texts[i])
    text = re.sub(r'이상입니다', '', text)
    text = re.sub(r'감사합니다', '', text)
    text = re.sub(r'답변이 도움이 되셨으면 합니다', '', text)
    text = re.sub(r'신홍범 저 이북스', '', text)
    text = re.sub(r'불면증 당신도 치료될 수 있다', '', text)
    text = re.sub(r'신홍범 저', '', text)
    text = re.sub(r'그럼에도 남들에게 말씀하시지도 못하고 두달이 넘는 긴 시간 동안 혼자 참아오시느라 많이 힘이 드셨겠네요', '', text)
    text = re.sub(r'알아두세요', '', text)
    text = re.sub(r'질문자의 정보 참조 용도로 활용 가능하며 정확한 개인 증상 파악은 의료 기관 내방을 통해 진행하시기 바랍니다', '', text)
    text = re.sub(r'포인트로 감사할 때 참고해주세요', '', text)
    text = re.sub(r'상담의사의 최종적인 의학적 소견을 나타낸 것이 아니며 답변을 제공한 개인 및 사업자의 법률적 책임이 없음을 알려드립니다', '', text)
    text = re.sub(r'시의적 물리적 환경에 따라 변경된 의학적 이견을 가질 수 있으며 의학적 소견이 상이한 여러 의견을 모두 수렴하지는 않습니다', '', text)
    text = re.sub(r'위 답변은 답변작성자가 경험과 지식을 바탕으로 작성한 내용입니다', '', text)
    text = re.sub(r'※ 의사 전문가의 예상 치료비 할인 가능 여부 등 추가 정보가 궁금하시면 아래 를 통해 확인할 수 있습니다', '', text)
    text = re.sub(r'※ 전문가 답변은 가격안내 홍보성 내용을 포함하지 않으며 원문은 를 통해 확인할 수 있습니다', '', text)
    text = re.sub(r'김상은 정신건강의학과 전문의 광주소망병원', '', text)
    text = re.sub(r'지식in 의사 답변은 대국민 건강 상담에 뜻을 둔 의사의 지식기부로서 인터엠디컴퍼니와 네이버의 제휴를 통해 의사가 직접 정확하고 신뢰할 수 있는 답변을 제공합니다', '', text)
    text = re.sub(r'도움이 되셨길 바랍니다', '', text)
    text = re.sub(r'질문하신 내용 잘 보았습니다', '', text)
    text = re.sub(r'공황장애 및 우울증에 대한 진단을 받으신 건가요', '', text)
    text = re.sub(r'안과 선생님 말씀을 들으신 대로 이해하시면 되겠습니다', '', text)
    text = re.sub(r'아래 그림 참고', '', text)
    text = re.sub(r'코골이 하지불안증후군 불면증 기면증에 대한 종합보고서', '', text)
    text = re.sub(r'북크', '', text)
    text = re.sub(r'싶습니다', '', text)
    text = re.sub(r'진짜 말기는', '', text)
    text = re.sub(r'어마어마하거든요', '', text)
    text = re.sub(r'krcounselviewcrtflxkhgelur', '', text)
    text = re.sub(r'krcounselviewcrwolxjgmxvm', '', text)
    text = re.sub(r'krcounselviewcrfzrwgbssu', '', text)
    text = re.sub(r'krcounselviewcrmxqijolljszx', '', text)
    text = re.sub(r'krcounselviewcrfzqygiqm', '', text)
    text = re.sub(r'krcounselviewcrmoniqlhhcej', '', text)

    # 임의 수정한 부분
    text = re.sub(r'자다가 중간에 회 혹은 그 이상 잠에서 깨는 문제로 인해 질문 하셨습니다', '자다가 중간에 자주 잠에서 깨는 문제로 인해 질문 하셨습니다', text)
    text = re.sub(r'불면증은의학적으로 잠 드는데까지 분 이상 소요되고그러한 증상이 주 이상 지속되면 불면증으로 진단 내릴 수 있습니다',
                  '불면증은 의학적으로 잠 드는데까지 30분 이상 소요되고그러한 증상이 3주 이상 지속되면 불면증으로 진단 내릴 수 있습니다', text)
    text = re.sub(r'학교에서 지내시는 중에 시도 때도 없이 수시로 참을 수 없는 정도로 잠이 쏟아진다”라고 말씀하셨습니다', '일상을 지내시는 중에 시도 때도 없이 수시로 참을 수 없는 정도로 잠이 쏟아진다고 말씀하셨습니다', text)
    text = re.sub(r'부모님에게 자세하게 상의를 해 보시고가까운 정신과병원에 상담을 받아보시면 도움이 분명 되실 겁니다', '주변의 믿을만한 분에게 자세하게 상의를 해 보시고 가까운 정신과병원에 상담을 받아보시면 도움이 분명 되실 겁니다', text)
    text = re.sub(r'최근 새벽 시시 사이에 깨는 증상을 지속적으로 겪고 계시는군요', '최근 새벽에 깨는 증상을 지속적으로 겪고 계시는군요', text)
    text = re.sub(r'그러나 지금 스스로 돌아보고 애를 쓰고 있는 상황에서 말기라고 할 것까지야', '그러나 지금 스스로 돌아보고 애를 쓰고 있는 상황에서 말기라고 할 것까지야 싶습니다', text)

    # 숫자 누락 (코드로 하는 게 더 나을거같아서 멈춤.. (역시 멍청하면 몸이 고생...아니야! 꼼꼼한거야!))
    text = re.sub(r'대 남성에서 심한 잠꼬대를 하는 질환으로렘수면행동장애가 있습니다', '중년 남성에게서 나타나는 심한 잠꼬대를 하는 질환으로 렘수면행동장애가 있습니다', text)
    text = re.sub(r'잠들기 전 분간의 반신욕을 통해 근육과 교감신경을 풀어 주면 숙면을 취할 수 있고침실을 건조하지 않게 하고 온도를 ∼도 내외로 조절하여 쾌적하게 만들면잠꼬대나 악몽을 꾸는 등의 증상을 줄일 수 있습니다', 
                  '잠들기 전 30분간의 반신욕을 통해 근육과 교감신경을 풀어 주면 숙면을 취할 수 있고 침실을 건조하지 않게 하고 온도를 18∼22도 내외로 조절하여 쾌적하게 만들면잠꼬대나 악몽을 꾸는 등의 증상을 줄일 수 있습니다', text)
    text = re.sub(r'누워서 잠들때까지 분이상 걸리고 이러한 증상이 주이상 지속되면 불면증을 의심합니다', '누워서 잠들때까지 30분이상 걸리고 이러한 증상이 3주이상 지속되면 불면증을 의심합니다', text)
    text = re.sub(r'실제로 유럽의 경우 발병에서 진단까지 년 정도가 걸린다고 하며그 동안 환자는 영문도 모르고 심한 졸음에 시달리게 됩니다', '실제로 유럽의 경우 발병에서 진단까지 7년 정도가 걸린다고 하며 그 동안 환자는 영문도 모르고 심한 졸음에 시달리게 됩니다', text)
    text = re.sub(r'이는 여년 전에 연구를 통해서 밝혀져 있으며 기면증에 대한 동물 연구에서도 확인되었습니다', '이는 10여년 전에 연구를 통해서 밝혀져 있으며 기면증에 대한 동물 연구에서도 확인되었습니다', text)
    
    removed_list.append(text)

    # print before, after
    if (i >= 100) and (i % 100 == 0):
      print("---"*15, i, "---"*15)
      print("[Before]: ", texts[i])
      print("[ After]: ", text)
      print("\n\n")

  return removed_list


"""
뒤에 온점이 없고 띄어쓰기만 있는 경우도 있어서 온점 포함을 안했기 때문에 또 제거해줘야함

인용구 처리 ???
너가 이렇게 의지가 약해서 걸렸다. 고 나무라지 않습니다.
"""

'\n뒤에 온점이 없고 띄어쓰기만 있는 경우도 있어서 온점 포함을 안했기 때문에 또 제거해줘야함\n\n인용구 처리 ???\n너가 이렇게 의지가 약해서 걸렸다. 고 나무라지 않습니다.\n'

In [ ]:
final_answers = custom_clean_answer(c_answers)

--------------------------------------------- 100 ---------------------------------------------
[Before]:  몽유병이라고 하는 것은 자다가 일어나서 걸어다니고 어떤 행동을 하고 말을하거나 이런양상을 말하는데요. 몽유병은 원래 소아에서 흔합니다. 예를 들어 세 특히 남자아이에서 흔하고 대게 이런 경우는 자라면서 저절로 좋아진다. 이렇게 많이 이야기하고 실제로도 그렇습니다. 성인인 경우는 이미 성장이 끝났는데도 몽유병이 있으니 자라면 좋아진다. 이런 말이 맞지가 않는 거고요. 이럴 경우에 몽유병과 관련된 여러 가지 위험이 따를 수 있으니 이에 따른 조치를 하는 것이 필요합니다. 특히 성인 몽유병이 있는 경우에 자다가 일어나 움직이다가 본인이나 주변 사람이 다칠 위험이 있습니다. 성인 몽유병의 경우에는 기면병과 같은다른 수면질환의 증상으로 나타나는 경우도 있을 수 있고 잠을 방해하는 수면 무호흡증 같은 질환이 있어서 나타나는 경우도 있습니다. 그래서 이런 상태가 일주일 이상 지속되면 수면 클리닉을 방문하셔서 진료를 받아보는 것이 필요합니다.
[ After]:  몽유병이라고 하는 것은 자다가 일어나서 걸어다니고 어떤 행동을 하고 말을하거나 이런양상을 말하는데요. 몽유병은 원래 소아에서 흔합니다. 예를 들어 세 특히 남자아이에서 흔하고 대게 이런 경우는 자라면서 저절로 좋아진다. 이렇게 많이 이야기하고 실제로도 그렇습니다. 성인인 경우는 이미 성장이 끝났는데도 몽유병이 있으니 자라면 좋아진다. 이런 말이 맞지가 않는 거고요. 이럴 경우에 몽유병과 관련된 여러 가지 위험이 따를 수 있으니 이에 따른 조치를 하는 것이 필요합니다. 특히 성인 몽유병이 있는 경우에 자다가 일어나 움직이다가 본인이나 주변 사람이 다칠 위험이 있습니다. 성인 몽유병의 경우에는 기면병과 같은다른 수면질환의 증상으로 나타나는 경우도 있을 수 있고 잠을 방해하는 수면 무호흡증 같은 질환이 있어서 나타나는 경우도 있습니다

# 5. 데이터 저장
완성된 answer와 그에 해당되는 question을 저장합니다.

In [ ]:
cd /content/drive/MyDrive/TobigSlang/'2. Modeling '/'정진모'/'미경'/models/'#preprocessing'

/content/drive/.shortcut-targets-by-id/1Tys5-QaX-3ICj4J5S283IJYXj7FSQFAe/TobigSlang/2. Modeling /정진모/미경/models/#preprocessing


In [ ]:
def save_data(question, answer, filename):
  
  #to dataframe
  dic = {'question': question, 'answer': answer}
  df = pd.DataFrame(data=dic)
  
  #drop na
  df = df.replace(r'^\s*$', np.nan, regex=True)
  df = df.dropna(axis=0).reset_index(drop=True)
  
  #to csv
  df.to_csv(filename+".csv", index=False)

  print("Saving", filename, "data is finished.")
  print("- 질문 개수 : ", len(question))
  print("- 답변 개수 : ", len(answer))
  print("- 완성된 데이터 프레임 : ", len(df))

  return df

In [ ]:
kin = save_data(final_questions, final_answers, "kin1228_pend")

Saving kin1228_pend data is finished.
- 질문 개수 :  33264
- 답변 개수 :  33264
- 완성된 데이터 프레임 :  33261


# 6. 데이터 불러오기 

In [ ]:
temp = pd.read_csv("kin1228_pend.csv", encoding="utf-8")
temp

,question,answer
0,아버지께서 코골이가 심하십니다. 코골이소리 엄청 큰 것은 물론이고요. 이번에 본집가...,수면무호흡증의 위험성과 관련해서 여러 가지 이슈가 있습니다. 우선은 충분한 잠을 못...
1,꿈을 안꾸는 날이 년에 손꼽을 정도이고 매일 꿈을 꿉니다. 꿈들이 다 기억이 나며 ...,저는 매일 밤 꿈을 꿉니다. 참 괴롭습니다. 이렇게 얘기 하시는 분이 있습니다. 이...
2,잠을 잘 못자요 잠들 때 까지도 시간이 오래걸리지만 잠들고 나서 새벽에 자꾸 깨서 ...,수면클리닉에 가면 모든 수면 질환을 다 진단할 수 있느냐 치료할 수 있느냐라고 궁금...
3,안녕하세요. 곧 살이 될 사람입니다. 다름이 아니라 제가 고등학생 때부터 몽유병인가...,몽유병의 치료 방법에 대해서 궁금해서 질문을 해주셨는데요. 우선 몽유병은 몽유병이 ...
4,불면증 약먹어도 괜찮을까요,불면증으로 고생하시는 분들이 있고요. 이분들이 수면제를 복용해 보신 분들도 있는데 ...
...,...,...
33259,제가 두달 전 공황발작 후 동네 신경정신과에서 처방받은 약을 세네번 먹었구요. 근데...,안녕하세요. 먼저 공황증상에 대해 설명하기 앞서서 불안장애의 큰 분류에 대해서 말씀...
33260,명인벤즈트로핀메실레이트정mg약성분좀 알려주세요,주로 뇌속의 콜린성수용체에 작용하여 항콜린성효과로 인해 파킨슨병에서 보일수있는 손떨...
33261,선천적으로 토를 너무 자주 하는 것 같습니다. 살 여자 술 담배는 전혀 안합니다. ...,잦은 구토 증세로 많이 힘드셨겠습니다. 선천적으로 토를 해왔다는 말씀이 아마도 질문...
33262,살 남자입니다. 일단 저는 제가 우울증이라 생각한적이 없습니다. 커오면서 외로움도 ...,마치 점점 추워지는 겨울처럼 알게 모르게 질문자님의 마음은 더 메마르고 힘겨워지고 ...


In [ ]:
for i in range(len(temp)):
  if i % 100 == 0:
    print("[question] ", temp['question'][i])
    print("[ answer ] ", temp['answer'][i])
    print("\n")


[question]  아버지께서 코골이가 심하십니다. 코골이소리 엄청 큰 것은 물론이고요. 이번에 본집가서 자는데 갑자기 코고는소리가 안나서 깜짝놀라서 가봤더니 막혔던 숨을몰아쉬면서 주무시더라고요. 코골이 수면무호흡증 인 것 같습니다. 그런데 문제는요즘들어 아버지께서 평상시에 심장통증 비슷하게 가슴 답답함을자주 느끼신다고 합니다. 쥐어짜는 느낌이라고 하시네요. 빨리 말씀하시지 제가 걱정할까봐 이제야 말씀하신 것 같습니다. 코골이만 생각했을땐 치료가 필요할까 생각했는데 코골이수면무호흡증이 심혈관 뇌혈관질환으로 이어질 가능성이 높다라는 기사를 우연히 보고 걱정이 됩니다. 더 정확한 답변을 듣고 싶어서 질문드리는데요. 수면무호흡증이 심뇌혈관질환과 관련이 있나요. 혈압도 좀 높으신 편입니다. 수면무호흡증치료 어떻게 하는 것이 좋을까요
[ answer ]  수면무호흡증의 위험성과 관련해서 여러 가지 이슈가 있습니다. 우선은 충분한 잠을 못 자니까 낮 동안에 졸리고 그래서 사고 위험이 높아진다가 있고요. 기억력이 떨어진다. 인지 기능이 떨어진다. 치매가 온다. 이런 것도 있습니다. 수면무호흡증과 관련된 가장 직접적이고 눈에 보이는 위협이라고 할 수 있는 것이혈압이 높아지면서 심장에 무리가 가고 그래서 부정맥 심근경색 심장마비 돌연사 이런 위험이 있는 거고요. 또 뇌혈관에 압력을 높여 가지고 뇌경색이라든지 뇌출혈이라든지 그래서 뇌혈관질환으로 인한 사망 위험을 높인다라고 되어 있습니다. 실제로 수면무호흡증 자체가 잠을 잘 때 숨이 멈추는 거고 우리 몸에 산소가 부족해지는 거고 그래서 산소를 많이 소모하는 심장과 뇌가 우선적으로 타격을 받는 거고요. 그 과정에서 뇌세포 심장세포가 죽어나가는 거고 그만큼 기능이 떨어지는 거고 그래서 사고가 생기는 겁니다. 혈압이 높아지는 거 그로 인한 혈관의 내피 부분에 손상이 생기고 혈관 암벽이 다치니까 그 부분이 치유되는 과정에서 두꺼워지죠막히게 되고 그리고 수면무호흡이 있을 때 심장이 상당히 불규칙하게 됩니다. 빨리 뛰다. 느리게 뛰다. 